In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [4]:
import pandas as pd
import sys
import tensorflow.compat.v1 as tf
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
yx = np.array(pd.read_csv("/kaggle/input/digit-recognizer/train.csv"))
X_test = np.array(pd.read_csv('/kaggle/input/digit-recognizer/test.csv'))
Y_train = yx[:, 0]
Y_train = np.transpose(np.matrix([Y_train]))
X_train = yx[:, 1:]

In [5]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, Y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=10)

In [ ]:
knn_y_pred = knn.predict(X_test)

In [ ]:
labels = [np.argmax(x) for x in knn_y_pred]
ids = [x+1 for x in range(len(knn_y_pred))]

sub = pd.DataFrame()

sub['ImageId'] = ids
sub['Label'] = labels

sub.to_csv('submission.csv', index=False)

In [ ]:
# Learning Curve 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
  
x_train = X_train[0:10000]
y_train = Y_train[0:10000]
  
# Obtain scores from learning curve function
# cv is the number of folds while performing Cross Validation
sizes, training_scores, testing_scores = learning_curve(knn, x_train, y_train, cv=5, scoring='accuracy')
  
# Mean and Standard Deviation of training scores
mean_training = np.mean(training_scores, axis=1)
Standard_Deviation_training = np.std(training_scores, axis=1)
  
# Mean and Standard Deviation of testing scores
mean_testing = np.mean(testing_scores, axis=1)
Standard_Deviation_testing = np.std(testing_scores, axis=1)
  
# dotted blue line is for training scores and green line is for cross-validation score
plt.plot(sizes, mean_training, '--', color="b",  label="Training score")
plt.plot(sizes, mean_testing, color="g", label="Cross-validation score")
  
# Drawing plot
plt.title("LEARNING CURVE FOR KNN Classifier")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
#### Multi Layer Perceptron


from sklearn.neural_network import MLPClassifier
mlpClassifier = MLPClassifier(hidden_layer_sizes=100, activation='relu', solver='adam', alpha=0.0001, batch_size='auto', 
             learning_rate='constant', 
              learning_rate_init=0.0009, power_t=0.5, max_iter=250, shuffle=True, random_state=None, tol=0.0001, 
              verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, 
              validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

# epsilon=1e-08 Value for numerical stability in adam. Only used when solver=’adam’
mlpClassifier.fit(X_train, Y_train)


In [ ]:
mlp_y_pred = mlpClassifier.predict(X_test)


In [ ]:
labels = [np.argmax(x) for x in mlp_y_pred]
ids = [x+1 for x in range(len(mlp_y_pred))]

sub = pd.DataFrame()

sub['ImageId'] = ids
sub['Label'] = labels

sub.to_csv('submission.csv', index=False)

In [ ]:
# Learning Curve for MLP
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes.set_title(title)
    if ylim is not None:
        axes.set_ylim(*ylim)
    axes.set_xlabel("Training examples")
    axes.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes.legend(loc="best")

    return plt


fig, axes = plt.subplots(1, 1)


title = "Learning Curves"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
#cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

plot_learning_curve(mlpClassifier, title, X_train, Y_train, axes=axes, ylim=(0.7, 1.01)
                    ,cv=3 
                    ,n_jobs=1)
# SVC is more expensive so we do a lower number of CV iterations:
#plt.savefig("MLP_learningCurve.png")
plt.show()

In [ ]:
## To get images near decision boundaries

#MLP 
# At image  3  Predicted as 0  with probability  0.5285171679951194 near probability of  9  is  0.4672753588286859
# At image  24907  Predicted as 1  with probability  0.5103879294802369 near probability of  9  is  0.4675327258484755
# At image  13200  Predicted as 2  with probability  0.5560217172706002 near probability of  7  is  0.4439782827289965
# At image  17205  Predicted as 3  with probability  0.5014700056696101 near probability of  8  is  0.4984282708620153
# At image  15440  Predicted as 4  with probability  0.4995766103290047 near probability of  7  is  0.4984102402548431
# At image  2297  Predicted as 5  with probability  0.5033862952747804 near probability of  4  is  0.4966137023580073
# At image  14401  Predicted as 6  with probability  0.4094020376969683 near probability of  5  is  0.2891308549880324
# At image  429  Predicted as 7  with probability  0.5066073567428863 near probability of  3  is  0.4933914979050861
# At image  1653  Predicted as 8  with probability  0.5058008324199895 near probability of  3  is  0.4941989510999071
# At image  3217  Predicted as 9  with probability  0.5028330820221076 near probability of  3  is  0.4971668625742563

MLP_y_pred_probab = mlp.predict_proba(X_test)

MLP_y_pred = np.array(mlp_y_pred)

k=1
for k in range(0,1):
    min = 1
    min_j = 0
    min_i=0
    for i in range(0,len(y_pred_probab)):
        for j in range(0, len(y_pred_probab[0])):
            if y_pred[i][1]==k and y_pred_probab[i][j] > 0.4 and y_pred[i][1] != j:
                diff = y_pred_probab[i][k] - y_pred_probab[i][j]
                if diff<min:
                    min = diff
                    min_i = i
                    min_j = j
                    print("# At image ",i," Predicted as", y_pred[i][1], " with probability ", y_pred_probab[i][k], "near probability of ", j, " is ", y_pred_probab[i][j])

